In [1]:
import numpy as np
import sklearn.ensemble
from anchor import utils
import xaibenchmark as xb
from xaibenchmark import load_adult as la
from xaibenchmark import preprocessing
from xaibenchmark.comparator import ExplainerComparator

np.random.seed(1)

done1
done2
Metric values for explainer ANCHORS , explanation created with the 0 instance of the given data
	 accuracy : 0.9375
	 balance_model : 0.0625
	 balance_explanation : 0
	 area_norm : 0.7937005259840998
	 balance_data : 0.07291666666666667
	 precision : 0.8043912175648703
	 area : 0.0625
	 inverse_coverage : 156.25
	 coverage : 0.0064
Metric values for explainer ANCHORS , explanation created with the 1 instance of the given data
	 accuracy : 0.9944567627494457
	 balance_model : 0.9944567627494457
	 coverage : 0.0709
	 inverse_coverage : 14.104372355430183
	 area : 0.3333333333333333
	 balance_explanation : 1
	 area_norm : 0.9125147547604939
	 balance_data : 0.9944567627494457
	 precision : 1.0


In [2]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = '../data/'
adult_dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)

In [3]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
rf.fit(adult_dataset.train, adult_dataset.labels_train)

RandomForestClassifier(n_estimators=50, n_jobs=5)

In [4]:
adult_dataset.__dict__.keys()

dict_keys(['labels', 'class_names', 'class_target', 'ordinal_features', 'categorical_features', 'categorical_names', 'feature_names', 'data', 'train', 'labels_train', 'validation', 'labels_validation', 'test', 'labels_test', 'test_idx', 'validation_idx', 'train_idx'])

In [5]:
adult_dataset.feature_names

['Age',
 'Workclass',
 'Education',
 'Marital Status',
 'Occupation',
 'Relationship',
 'Race',
 'Sex',
 'Capital Gain',
 'Capital Loss',
 'Hours per week',
 'Country']

In [6]:
adult_dataset.train

array([[ 3.,  6., 15., ...,  0.,  1., 39.],
       [ 2.,  7., 10., ...,  0.,  0., 22.],
       [ 3.,  4., 15., ...,  0.,  0., 39.],
       ...,
       [ 1.,  6.,  5., ...,  0.,  1., 39.],
       [ 3.,  4., 11., ...,  0.,  0., 39.],
       [ 2.,  4., 15., ...,  0.,  0.,  8.]])

In [7]:
data = la.load_csv_data('adult', root_path=dataset_folder)
lime_training_set = preprocessing.lime_preprocess_dataset(data.data, data.categorical_features, data.data.keys())
lime_ml_model = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
lime_ml_model.fit(lime_training_set, data.target.to_numpy().reshape(-1))    

RandomForestClassifier()

--------------
### IMport Explainers

In [8]:
from xaibenchmark.explainers import AnchorsExplainer, LimeExplainer

### Usage of implemented explainer

In [9]:
# instantiate anchors explainer
exp1 = AnchorsExplainer(rf, '../data/', 'adult')
exp2 = LimeExplainer(data, lime_ml_model, discretize_continuous=False)

In [10]:
data = la.load_csv_data('adult', root_path='../data/')

In [11]:
explanation = exp1.explain_instance(data.data.iloc[[70]], "test", threshold=0.99)
print("Current explanation:", explanation.names())

Current explanation: ['Education = 5th-6th', 'Relationship = Own-child', 'Hours per week <= 40.00', 'Capital Gain = 0']
TEST <class 'set'>


C:\Users\timri\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\timri\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
for item in exp1.report():
    for el in item:
        print(type(el))

<class 'str'>
<class 'float'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'numpy.int32'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'numpy.float64'>


In [13]:
comp = ExplainerComparator()
comp.add_explainer(exp1, 'ANCHORS')
comp.add_explainer(exp2, 'LIME')

In [14]:
comp.explain_instances(data.data.iloc[[1, 2, 3000]])

In [15]:
comp.print_metrics()

Average metric values for explainer ANCHORS :
	 accuracy : 0.8440258594682511
	 coverage : 0.0146
	 balance_model : 0.15597414053174885
	 inverse_coverage : 88.43671934991592
	 balance_explanation : 0.0
	 area_norm : 0.7937005259840998
	 balance_data : 0.15597045147496688
	 area : 0.0625
	 precision : 0.7934397417960226
Average metric values for explainer LIME :
	 furthest_distance : 5.539484077519858
	 area_norm : 19.537233873073223
	 accuracy : 0.8065487942204479
	 inverse_coverage : 16908.766072836985
	 coverage : 7.633831617481052e-05
	 balance : 0.07437956097632314
	 area : 2.589477453233665e+139


In [16]:
comp.print_metrics(explainer="ANCHORS", index=1)
comp.print_metrics(explainer="LIME", index=1)

Metric values for explainer ANCHORS , explanation created with the 1 instance of the given data
	 accuracy : 0.9375
	 coverage : 0.0064
	 balance_model : 0.0625
	 balance_explanation : 0
	 area_norm : 0.7937005259840998
	 balance_data : 0.07291666666666667
	 area : 0.0625
	 inverse_coverage : 156.25
	 precision : 0.815230961298377
Metric values for explainer LIME , explanation created with the 1 instance of the given data
	 area_norm : 19.537233873073223
	 accuracy : 0.8010354056069846
	 furthest_distance : 14.38285723485059
	 coverage : 0.00013663197251347076
	 balance : 0.06313341815469661
	 inverse_coverage : 7318.93115208747
	 area : 2.589477453233665e+139
